In [12]:
'''
A traves del uso de BLAST, se obtendria el accesion number de la secuencia proporcionada en el fichero gene.fasta.
Podeis utilizar cualquiera de las alternativas de uso de BLAST estudiadas en el Notebook 2B de la asignatura
(en local o contra el servidor de NCBI).
Almacenad el resultado obtenido con BLAST en el fichero que se ha indicado en el parametro --output.
Como seleccionar el accesion number del hit adecuado:
• Deberia tener un e-value menor de 0.01
• Entre los que cumplan la condicion anterior seleccionaremos el que mayor score tenga.
'''
from Bio.Blast import NCBIWWW, NCBIXML
from Bio import SeqIO

#leemos la secuencia del fichero gene.fasta
registro= SeqIO.read("data/gene.fasta", "fasta")

# ejecutamos el BLAST
blast_result = NCBIWWW.qblast("blastp", "nr", registro.seq, expect=0.01, hitlist_size=10)

# Guardamos el resultado en un fichero 
with open("data/salida.xml", "w") as archivo_blast:
    archivo_blast.write(blast_result.read())


# Cerramos el resultado del BLAST
blast_result.close()

# Leemos el resultado del BLAST
# queremos el accession number del hit adecuado y que mayor score tenga

with open("data/salida.xml") as archivo_blast:
    blast_record = NCBIXML.read(archivo_blast)

    # Inicializamos el mejor hit
    mejor_hit = None
    mejor_score = 0

    # Iteramos sobre los hits
    for hit in blast_record.alignments:
        for hsp in hit.hsps:
            if hsp.expect < 0.01 and hsp.score > mejor_score:
                mejor_hit = hit
                mejor_score = hsp.score


    # Guardar el accession number del mejor hit en un archivo de salida
    with open("data/salida.txt", "w") as salida:
        if mejor_hit:
            salida.write(f"Accession number: {mejor_hit.accession}\n")
        else:
            salida.write("No se encontraton\n")

    from Bio import Entrez

print("Accession number:", mejor_hit.accession)

Accession number: AAB51347


In [ ]:
Entrez.email="raul.ortegare@gmail.com"
Entrez.api_key="90fba815d4e3b0c6891f4558feedd9ad5e09"
accession = mejor_hit.accession

handle = Entrez.efetch(db = "protein", id = accession, rettype = "gb", retmode = "text")
record = SeqIO.read(handle, "genbank")
handle.close()
print(record.seq)

print(record.name)  # no se si usar .name o .id o el accession tal cual
print(record)

MTHLERSRQMSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTFSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK
AAB51347
ID: AAB51347.1
Name: AAB51347
Description: green fluorescent protein (mitochondrion) [synthetic construct]
Number of features: 4
/topology=linear
/data_file_division=SYN
/date=26-JUL-2016
/accessions=['AAB51347']
/sequence_version=1
/db_source=locus SCU89685 accession U89685.1
/keywords=['']
/source=mitochondrion synthetic construct
/organism=synthetic construct
/taxonomy=['other sequences', 'artificial sequences']
/references=[Reference(title='FACS-optimized mutants of the green fluorescent protein (GFP)', ...), Reference(title='Expression of green fluorescent protein from a recoded gene inserted into Saccharomyces cerevisiae mitochondrial DNA', ...), Reference(title='Direct Submission', ...)]
/molecule_type=protein
Seq('MTHLERSRQMS

In [30]:
def smith_waterman(seq1, seq2, matrix_name, gap_penalty):
    """
    Implementación del algoritmo Smith-Waterman.

    Args:
        seq1 (str): Primera secuencia.
        seq2 (str): Segunda secuencia.
        matrix_name (str): Nombre de la matriz de sustitución.
        gap_penalty (int): Penalización por huecos.

    Returns:
        tuple: (alineamiento seq1, alineamiento seq2, puntuación)
    """
    matrix = load(matrix_name)
    n = len(seq1) + 1
    m = len(seq2) + 1
    score_matrix = np.zeros((n, m), dtype=int)
    traceback = np.zeros((n, m), dtype=str)

    max_score = 0
    max_pos = (0, 0)

    for i in range(1, n):
        for j in range(1, m):
            match = matrix[seq1[i - 1], seq2[j - 1]]
            diag = score_matrix[i-1][j-1] + match
            up = score_matrix[i-1][j] + gap_penalty
            left = score_matrix[i][j-1] + gap_penalty
            score_matrix[i][j] = max(0, diag, up, left)

            if score_matrix[i][j] == diag:
                traceback[i][j] = 'D'
            elif score_matrix[i][j] == up:
                traceback[i][j] = 'U'
            elif score_matrix[i][j] == left:
                traceback[i][j] = 'L'
            else:
                traceback[i][j] = '0'

            if score_matrix[i][j] >= max_score:
                max_score = score_matrix[i][j]
                max_pos = (i, j)
    # Backtracking
    i, j = max_pos
    aligned1, aligned2 = [], []

    while score_matrix[i][j] != 0:
        if traceback[i][j] == 'D':
            aligned1.append(seq1[i - 1])
            aligned2.append(seq2[j - 1])
            i -= 1
            j -= 1
        elif traceback[i][j] == 'U':
            aligned1.append(seq1[i - 1])
            aligned2.append('-')
            i -= 1
        elif traceback[i][j] == 'L':
            aligned1.append('-')
            aligned2.append(seq2[j - 1])
            j -= 1

    aligned1.reverse()
    aligned2.reverse()

    return ''.join(aligned1), ''.join(aligned2), max_score

In [35]:
aligned1, aligned2, score_sw = smith_waterman(registro.seq, record.seq, "BLOSUM62", -5)
print("Alineamiento Smith-Waterman:")
print(aligned1)
print(aligned2)
print("Puntuación:", score_sw)

Alineamiento Smith-Waterman:
MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFIYTTGKLPVPWPTLVTTFSYGVQCFSRFPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEIKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSIRLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK
MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTFSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK
Puntuación: 1254
